# Stixrude-Lithgow-Bertelloni pseudo-omnicomponent phase generation
Required Python packages/modules

In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
import scipy.linalg as lin 
import scipy as sp
import sys
import sympy as sym

import matplotlib.pyplot as plt

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

In [ ]:
def get_subsolidus_phases(database='Berman'):
    remove_phases = ['Liq','H2O']
    
    modelDB = model.Database(database)
    phases = modelDB.phases
    [phases.pop(phs) for phs in remove_phases]
        
    return phases
        
def system_energy_landscape(T, P, phases, TOL=1e-3):
    elem_comps = []
    phs_sym = []
    endmem_ids = []
    mu = []
    for phsnm in phases:
        phs = phases[phsnm]
        
        elem_comp = phs.props['element_comp']
        abbrev = phs.abbrev
        endmem_num = phs.endmember_num
        iendmem_ids = list(np.arange(endmem_num))
        
        if phs.phase_type=='pure':
            nelem = np.sum(elem_comp)
            mu += [phs.gibbs_energy(T, P)/nelem]
            # print(nelem)
        else:
            nelem = np.sum(elem_comp,axis=1)
            # print(nelem)
            for i in iendmem_ids:
                imol = np.eye(phs.endmember_num)[i]
                mu += [phs.gibbs_energy(T, P, mol=imol,deriv={"dmol":1})[0,i]/nelem[i]]
                # print(nelem[i])
                
        endmem_ids.extend(iendmem_ids)
        phs_sym.extend(list(np.tile(abbrev,endmem_num)))
        # print(elem_comp)
        
        elem_comps.extend(elem_comp)
        # print(elem_comp)
        # print(phs)
        
    elem_comps = np.vstack(elem_comps)
    
    natoms = np.sum(elem_comps,axis=1)
    elem_comps = elem_comps/natoms[:,np.newaxis]
    
    elem_mask = ~np.all(elem_comps<TOL, axis=0)
    
    elem_comps = elem_comps[:, elem_mask]
    mu = np.array(mu)
    endmem_ids = np.array(endmem_ids)
    
    sys_elems = core.chem.PERIODIC_ORDER[elem_mask]
    return phs_sym, endmem_ids, mu, elem_comps, sys_elems

def prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps, decimals=4):
    elem_round_comps = np.round(elem_comps, decimals=decimals)
        # Drop identical comps
    elem_comps_uniq = np.unique(elem_round_comps, axis=0)
    
    # uniq_num = elem_comps_uniq.shape[0]
    mu_uniq = []
    phs_sym_uniq = []
    endmem_ids_uniq = []
    for elem_comp in elem_comps_uniq:
        is_equiv_comp = np.all(elem_round_comps == elem_comp[np.newaxis,:], axis=1)
        equiv_ind = np.where(is_equiv_comp)[0]
        min_ind = equiv_ind[np.argsort(mu[equiv_ind])[0]]
        min_mu = mu[min_ind]
        assert np.all(min_mu <= mu[equiv_ind]), 'fail'
        
        mu_uniq.append(min_mu)
        phs_sym_uniq.append(phs_sym[min_ind])
        endmem_ids_uniq.append(endmem_ids[min_ind])
        
    mu_uniq = np.array(mu_uniq)
    phs_sym_uniq = np.array(phs_sym_uniq)
    elem_comps_uniq = np.array(elem_comps_uniq)
    
    return phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq



### T,P, parameters and options for pseudo-phase generation

In [ ]:
T = 1300.0                  # K
P = 5000.0                 # bars
P = 300000.0                 # bars
# T = 1500.0                  # K
# P = 40000.0                 # bars

In [ ]:
database='Berman'
database='Stixrude'

In [ ]:
test_endmember_code = False # output tests to validate solution endmember code generation
test_solution_code = False  # output tests to validate solution code generation
offset_value = 0.0          # Offset penality (in J) to destabilize pseudo-omnicomponent phase 4000
use_oxides_as_basis = False # Construct the pseudo-phase using oxides as components (False == elements)

In [ ]:
phases = get_subsolidus_phases(database=database)
# ADD extra phases (e.g. carbonates as needed here)

In [ ]:
phs_sym, endmem_ids, mu, elem_comps, sys_elems = system_energy_landscape(T, P, phases)
# display(phs_sym, endmem_ids, mu, elem_comps, sys_elems)
phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq = (
    prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps))
Nelems = len(sys_elems)
Npts = mu_uniq.size

In [ ]:
sys_elems

In [ ]:
comps = elem_comps_uniq
mu = mu_uniq

In [ ]:
def min_energy_assemblage(bulk_comp, comp, mu, TOLmu=10, TOL=1e-5):
    xy = np.hstack((comp, mu[:,np.newaxis]))
    yavg = np.mean(mu)
    xy_bulk = np.hstack((bulk_comp, yavg))
    
    wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
    # print('rnorm',rnorm0)
    
    
    def fun(mu, shift=0):
        xy_bulk[-1] = mu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        return rnorm-shift
    
    
    delmu = .1
    if rnorm0==0:
        shift_dir = -1
        soln_found = True
    else:
        output = opt.minimize_scalar(fun, bounds=[np.min(mu), np.max(mu)])
        xy_bulk[-1] = output['x']
        wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
        shift_dir = -1
        
    mu_prev=xy_bulk[-1]
    rnorm=rnorm0
    
    while True:
        mu_prev = xy_bulk[-1]
        rnorm_prev = rnorm
        
        xy_bulk[-1] += shift_dir*delmu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        delmu *= 2
        
        # print(shift_dir, rnorm)
        if ((shift_dir==+1)&(rnorm>rnorm_prev)) or ((shift_dir==-1)&(rnorm>0)):
            break
            
        
    fun_fit = lambda mu, TOL=TOL: fun(mu, shift=TOL)
    if rnorm > TOL:
        mu_bulk = opt.brentq(fun_fit, mu_prev, xy_bulk[-1], xtol=TOLmu)
        xy_bulk[-1] = mu_bulk
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        
    mu_bulk = xy_bulk[-1]
    wt_bulk = wt
        
        
    ind_assem = np.where(wt_bulk>0)[0]
    return wt_bulk, mu_bulk, ind_assem 


# Define bulk composition

In [ ]:
wt = np.random.rand(elem_comps_uniq.shape[0])
wt = wt/np.sum(wt)
bulk_comp = np.dot(wt, elem_comps_uniq)

# Get minimum energy assemblage

In [ ]:
wt_bulk, mu_bulk, ind_assem = min_energy_assemblage(
    bulk_comp, elem_comps_uniq, mu_uniq, TOLmu=10)
# print(len(ind_assem))

np.sum(wt_bulk)
np.sum(wt_bulk>0)

In [ ]:
hull_phs_sym = np.unique(phs_sym_uniq[ind_assem])
hull_phs_sym

In [ ]:
# np.dot(wt_bulk,mu_uniq)

In [ ]:
# mu_bulk/1e3

# Determine hull phases

In [ ]:
def get_hull_pts(comps, mu, ind_assem, debug=True, TOL=1e-4, Nelems=Nelems):
    comp_hull = comps[ind_assem,:]
    mu_hull = mu[ind_assem]
    ind_hull = ind_assem.copy()
    
    Nphs = len(mu)
    mask = np.tile(True, Nphs)
    mask[ind_assem] = False
    
    mu_extra = mu[mask]
    comp_extra = comps[mask,:]
    ind_extra = np.arange(Nphs)[mask]
    
    for icomp, imu, ind in zip(comp_extra, mu_extra, ind_extra):
        
        # for ind in inds[Nhull_pts:]:
        iwt_hull, wt_resid = sp.optimize.nnls(comp_hull.T,icomp,maxiter=1000)
        icomp_mod = np.dot(iwt_hull, comp_hull) 
        imu_mod = np.dot(iwt_hull, mu_hull)
        
        icomp_resid = icomp-icomp_mod
        comp_match = np.all(np.abs(icomp_resid)<TOL)
        imaxdev = np.max(np.abs(icomp_resid))
        lower_energy = imu< imu_mod
        
        add_pt = (not comp_match) or (comp_match and lower_energy)
        if debug:
            print(add_pt, '---', len(mu_hull),
                  '(',comp_match,' , ', lower_energy, ') : ', imaxdev)
        if add_pt:
            comp_hull = np.vstack((comp_hull, icomp))
            mu_hull = np.hstack((mu_hull, imu))
            ind_hull = np.hstack((ind_hull, ind))
            
    return comp_hull, mu_hull, ind_hull

In [ ]:
comp_hull, mu_hull, ind_hull = get_hull_pts(
    elem_comps_uniq, mu_uniq, ind_assem, debug=False, TOL=1e-4, Nelems=Nelems)

In [ ]:
ind_hull

In [ ]:
def get_quad_inds(Nelems):
    ind_rows, ind_cols = np.tril_indices(Nelems,-1)
    cross_term_inds = np.vstack((ind_rows,ind_cols))
    return cross_term_inds

In [ ]:
def eval_curv(comps, method, cross_term_inds):
    single_pt = False
    if comps.ndim==1:
        single_pt = True
        comps = comps[np.newaxis,:]
        
    if method=='quad':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        X2_sum = np.sum(XiXj,axis=1)
        curv_term = X2_sum
    elif method=='quad-full':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        curv_term = XiXj
    elif method=='xlogx':
        logX = np.log(comps)
        logX[comps==0] = 0
        XlogX = comps*logX
        # XlogX[comps==0] = 0
        XlogX_sum = np.sum(XlogX,axis=1)
        curv_term = XlogX_sum
    else:
        assert False, method + ' is not a valid method for eval_curv.'
        
    if single_pt:
        curv_term = curv_term[0]
    
    return curv_term

In [ ]:
cross_term_inds = get_quad_inds(Nelems)

# Add linear combo points to flesh out hull

In [ ]:
N_hull = comp_hull.shape[0]
mix_cross_ind = np.vstack(np.tril_indices(N_hull, -1))

comp_mix = 0.5*comp_hull[mix_cross_ind[0]]+0.5*comp_hull[mix_cross_ind[1]]
mu_mix= 0.5*mu_hull[mix_cross_ind[0]] + 0.5*mu_hull[mix_cross_ind[1]]


In [ ]:
comp_full = np.vstack((comp_hull,comp_mix))
mu_full = np.hstack((mu_hull,mu_mix))

In [ ]:
def init_lstsq_xobs(comps, mu, curv_method, cross_term_inds=cross_term_inds):
    curv_term = eval_curv(comps, curv_method, cross_term_inds)
    if curv_term.ndim==1:
        curv_term = curv_term[:,np.newaxis]
        
    xobs = np.hstack((comps, curv_term))
    yexp_scl = np.floor(np.log10(np.max(mu)-np.min(mu)))
    yscl = 10**yexp_scl
    yobs = mu/yscl
    
    return xobs, yobs, yscl

In [ ]:
# curv_term = eval_curv(comps, 'quad-full', cross_term_inds)
# xlogx = eval_curv(comps, 'xlogx', None)
# quad = eval_curv(comps, 'quad', cross_term_inds)

In [ ]:
curv_method = 'quad-full'
curv_method = 'quad'
# xobs, yobs, yscl = init_lstsq_xobs(comps, mu, curv_method)
xobs, yobs, yscl = init_lstsq_xobs(comp_full, mu_full, curv_method)

In [ ]:
yobs
yscl

In [ ]:
def reweight_fit(scl, xobs, yobs, bound='lower', yresid=0, TOL=1e-4, Nelems=Nelems):
    if np.isscalar(yresid):
        yresid = np.tile(yresid, yobs.size)
        
    err = np.ones(yobs.shape)
    mask_pos = yresid>0
    mask_neg = yresid<0
    
    yabs_dev = np.abs(yresid)
    err0 = np.median(yabs_dev)
    if err0==0:
        err0=1
    
    yabs_dev[yabs_dev<TOL] = TOL
    
    err_fac = 1/np.sqrt(yabs_dev)
    
    # err[mask_pos] = err0*scl*err_fac[mask_pos]
    # err[mask_neg] = err0/scl*err_fac[mask_neg]
    if bound=='lower':
        err[mask_pos] = err0/scl*err_fac[mask_pos]
        err[mask_neg] = err0*scl*err_fac[mask_neg]
    elif bound=='upper':
        err[mask_pos] = err0/scl*err_fac[mask_pos]
        err[mask_neg] = err0*scl*err_fac[mask_neg]
    else:
        assert False, 'bound not valid.'
    
    xobs_wt = xobs/err[:, np.newaxis]
    yobs_wt = yobs/err
    
    
    
    # wt_fit = np.linalg.lstsq(xobs_wt, yobs_wt, rcond=None)
    # param_wt = wt_fit[0]
    
    Nparams = xobs.shape[1]
    Ncurv = Nparams-Nelems
    # lowbnd = np.hstack((-10, np.tile(-np.inf, N), np.tile(0, N)))
    # hibnd = np.hstack((+10, np.tile(+np.inf, N), np.tile(+np.inf, N)))
    
    lowbnd = np.tile(-np.inf, Nparams)
    hibnd = np.tile(+np.inf, Nparams)
    # lowbnd[-Ncurv:] = 0
    hibnd[-Ncurv:] = 0
    
    # lowbnd = np.hstack((np.tile(-np.inf, Nelems), np.tile(0, Ncurv) ))
    # hibnd = np.hstack((np.tile(+np.inf, Nelems), np.tile(+np.inf, Ncurv)))
    
    # lowbnd = np.hstack((np.tile(-np.inf, N), np.tile(-np.inf, N)))
    
    # bnds = []
    # for ihi, ilo in zip(hibnd, lowbnd):
    #     bnds.append((ilo, ihi))
    
    # N = int(xobs.shape[1]/2)
    # lowbnd = np.hstack((np.tile(-np.inf, N), np.tile(0, N)))
    # hibnd = np.hstack((np.tile(+np.inf, N), np.tile(+np.inf, N)))
    
    wt_fit = opt.lsq_linear(xobs_wt, yobs_wt, bounds=(lowbnd, hibnd)) 
    # fun = lambda params, x=xobs_wt, y=yobs_wt: np.sum((y-np.dot(x, params))**2)
    # if param0 is None:
    #     param0 = -np.ones(2*Ndim)
    #     
    # # print(x0)
    # # print(fun(x0))
    # wt_fit = opt.minimize(fun, param0, bounds=bnds) 
    param_wt = wt_fit['x']
    
    
    
    
    yresid_wt = yobs -  np.dot(xobs, param_wt)
    
    return param_wt, yresid_wt, err
    
def plot_resid(ind, yresid, comp, err=None, xlim=(-0.1,1.1)):
    plt.figure()
    icomp = comp.T[ind]
    
    # xmax = np.max((np.abs(np.min(icomp)), np.abs(np.max(icomp))))
    # x = np.linspace(-1.1*xmax, +1.1*xmax, 101)
    x = np.linspace(xlim[0], xlim[1], 101)
    if err is None:
        plt.plot(icomp, yresid, 'ko')
    else:
        plt.errorbar(icomp, yresid, yerr=err, fmt='ko')
        
    plt.plot(x,0*x,'r--')
    plt.xlim(xlim)
    
def energy_diff(wt, yobs, comp, param_wt, curv_method='quad', 
                cross_term_inds=cross_term_inds):
    comp_wt = np.dot(comp.T, wt)
    
    N = comp.shape[1]
    mu_pseudo_lin = np.dot(param_wt[:N], comp_wt)
    
    curv_term = eval_curv(comp_wt, method=curv_method, 
                          cross_term_inds=cross_term_inds)
    mu_pseudo_curv = param_wt[-1]*curv_term
    
    mu_endmem = np.dot(wt, yobs)
    
    dmu = mu_pseudo_lin + mu_pseudo_curv - mu_endmem
    return dmu

def endmem_subset(mask, yobs, comp):
    comp_sub = comp[mask,:]
    yobs_sub = yobs[mask]
    
    return yobs_sub, comp_sub
    
    
# def energy_jac(wt, dmu_lin, quad_terms):
#     # dmu_lin = np.dot(param_wt[:NX],comp_eig.T)
#     dmu_quad = np.dot(wt, quad_terms)
#     # dmu_endmem = yobs
#     
#     dmu_dw =  dmu_lin+dmu_quad
#     return dmu_dw

In [ ]:
def fit_bound(param_wt, yresid_wt, xobs, yobs, 
              bound='lower', hull_thresh=1e3, expfac_inc=0.1):
    fac=0
    while True:
        fac += expfac_inc
        param_wt, yresid_wt, err = reweight_fit(
            10**fac, xobs, yobs, bound=bound, yresid=yresid_wt)
        
        if bound=='lower':
            mask = yresid_wt<0
        else:
            mask = yresid_wt>0
            
        err_bnd = np.sqrt(np.mean(yresid_wt[mask]**2))
        if err_bnd*yscl < 0.3*hull_thresh:
            break
            
    return param_wt, yresid_wt, err


# Initial Fit

In [ ]:
param0, yresid0, err0 = reweight_fit(1.0, xobs, yobs, yresid=0)


In [ ]:
comp_dev = np.sum((comp_full-bulk_comp)**2, axis=1)
# comp_dev

In [ ]:
plt.figure()
plt.plot(comp_dev, yscl*yresid0,'ko')
plt.plot(comp_dev[ind_assem], yscl*yresid0[ind_assem],'rx', ms=10, mew=3)
plt.ylabel('G [J]')

In [ ]:
yscl

# Fit upper bound on points

In [ ]:
bound='upper'
# bound='lower'
param, yresid, err = fit_bound(param0, yresid0, xobs, yobs,bound=bound)

In [ ]:
plt.figure()
plt.plot(comp_dev, yscl*yresid,'ko')
plt.plot(comp_dev[ind_assem], yscl*yresid[ind_assem],'rx', ms=10, mew=3)

In [ ]:
param[Nelems:]

In [ ]:
## Shift upwards to obey all points1-

In [ ]:
mu_uniq

In [ ]:
dmu = yscl*np.max(yresid)
dmu

In [ ]:
param[:Nelems]+= dmu/yscl

In [ ]:
yresid = yobs - np.dot(xobs, param)

In [ ]:
plt.figure()
plt.plot(comp_dev, yscl/1e3*yresid,'ko')
plt.plot(comp_dev[ind_assem], yscl/1e3*yresid[ind_assem],'rx', ms=10, mew=3)
plt.plot([0,np.max(comp_dev)], [0,0], 'r--')
# plt.ylim(-15,1)

In [ ]:
# np.dot(xobs, yscl*param)

In [ ]:
len(param)
param[Nelems:]

In [ ]:
mu_curv = param[Nelems:]
dlogmu = 2
thresh = np.max(np.log10(mu_curv)) - dlogmu
ind = np.arange(len(mu_curv))
mask = mu_curv<thresh

plt.figure()
plt.semilogy(ind, mu_curv,'ko')
plt.semilogy(ind[mask],mu_curv[mask],'rx',mew=3, ms=10)

# mu_curv[mask] = thresh

In [ ]:
mu_curv

In [ ]:
curv_mat = np.zeros((Nelems,Nelems))

curv_mat[cross_term_inds[0], cross_term_inds[1]] = mu_curv
curv_mat[cross_term_inds[1], cross_term_inds[0]] = mu_curv
# curv_mat[cross_term_inds[0], cross_term_inds[1]] = 1
# curv_mat[cross_term_inds[1], cross_term_inds[0]] = 1
# for i,j in cross_term_inds.T:
#     curv_mat[i, j] = mu_curv
np.linalg.det(curv_mat)

In [ ]:
plt.imshow(curv_mat, cmap='viridis')
plt.colorbar()

In [ ]:
# param[Nelems:] = mu_curv
# yresid = yobs - np.dot(xobs, param)

In [ ]:
# plt.figure()
# plt.plot(comp_dev, yscl/1e3*yresid,'ko')
# plt.plot(comp_dev[ind_assem], yscl/1e3*yresid[ind_assem],'rx', ms=10, mew=3)
# plt.plot([0,np.max(comp_dev)], [0,0], 'r--')
# plt.ylim(-15,1)

In [ ]:
# mu_local, curv_local, mu_resid = fit_local_model(
#     bulk_comp, mu_bulk, ind_assem, elem_comps_uniq, mu_uniq, 
#     mu_TOL=5e3, display_fit=True)
# # plt.ylim(-300,30)
# print(mu_local/1e3)
# print(curv_local/1e3)

In [ ]:
# bulk_comp, mu_bulk, ind_assem, comps, mu = (
#     bulk_comp, mu_bulk, ind_assem, elem_comps_uniq, mu_uniq
# )
# mu_TOL=10

In [ ]:
#     
# comp_assem = comps[ind_assem,:]
# mu_assem = mu[ind_assem]
# 
# output = np.linalg.lstsq(comp_assem, mu_assem, rcond=None)
# mu_local = output[0]
# 
# mu_local += mu_bulk-np.dot(mu_local, bulk_comp) + mu_TOL
# 
# 
# comp_dev = comps-bulk_comp
# curv_term = np.sum(comp_dev**2, axis=1)
# 
# dmu = mu-np.dot(comps, mu_local)
# 
# mask = np.tile(True, len(mu))
# mask[ind_assem] = False
# 
# 
# x = curv_term[mask, np.newaxis]
# y = dmu[mask, np.newaxis]
# 
# output = np.linalg.lstsq(x, y, rcond=None)
# curv_local = output[0][0][0]
# 
# mu_resid = dmu - curv_local*curv_term
# 
# 

In [ ]:
# hull_thresh = 1e3
# # %%timeit
# param0, yresid0, err0 = reweight_fit(1.0, xobs, yobs, yresid=0)
# param_wt, yresid_wt, err = fit_lower_bound(param0, yresid0, xobs, yobs, hull_thresh)


In [ ]:
# plt.figure()
# plt.plot(yresid_wt,'ko')

In [ ]:
# np.sum(curv_term>0,axis=0)

In [ ]:
# # x = curv_term[mask, np.newaxis]
# # y = dmu[mask, np.newaxis]
# x = np.hstack((comps, curv_term, xlogx[:,np.newaxis]))
# # x = np.hstack((comps, xlogx[:,np.newaxis]))
# # x = np.hstack((comps, quad[:,np.newaxis]))
# y = mu[:, np.newaxis]


In [ ]:
# output = np.linalg.lstsq(x, y, rcond=None)

In [ ]:
# params = np.squeeze(output[0])

In [ ]:
# params[Nelems:]

In [ ]:
# params.shape

In [ ]:
# resid = mu-np.dot(x,params)

In [ ]:
# plt.figure()
# plt.plot(resid/1e3,'ko')
# plt.plot([0, len(resid)],[0,0],'r--')

In [ ]:
# plt.figure()
# plt.plot(resid/1e3,'ko')
# plt.plot([0, len(resid)],[0,0],'r--')

In [ ]:
# # xobs = np.hstack((comps, curv_term[:, np.newaxis]))
# 
# comp_dist = np.sqrt(curv_term)
# plt.figure()
# plt.plot(comp_dist, mu_resid/1e3,'ko')
# plt.plot(comp_dist[ind_assem], mu_resid[ind_assem]/1e3,'rx')
# plt.plot([0, np.max(comp_dist)], [0,0],'r--')
# print(mu_resid[ind_assem]/1e3)
# print(comp_dist[ind_assem])

In [ ]:
# %%timeit
# 
# wt_bulk, mu_bulk, ind_assem = min_energy_assemblage(bulk_comp, elem_comps_uniq, mu_uniq, TOLmu=10)
# mu_local, curv_local = fit_local_model(
#     bulk_comp, mu_bulk, ind_assem, elem_comps_uniq, mu_uniq)

## Build endmembers of pseudo-phase using the coder module

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
def standardize_formula(form):
    cmp = form.split('O')
    str = ''
    if cmp[0][-1].isdigit():
        str += cmp[0][:-1] + '(' + cmp[0][-1] + ')'
    else:
        str += cmp[0] + '(1)'
    if cmp[1] == '':
        str += 'O'
    else:
        str += 'O(' + cmp[1] + ')'
    return str

In [ ]:
# param

In [ ]:
mu_linear = yscl*param[:Nelems]
mu_curv = yscl*param[Nelems:]

In [ ]:
model_type = "calib"
for ind,elm in enumerate(sys_elems):
    imu = mu_linear[ind]
    if use_oxides_as_basis:
        formula = standardize_formula(elm)
    else:
        formula = elm+'(1)'
    param_dict = {'Phase':elm,'Formula':formula,'T_r':298.15,'P_r':1.0,'GTP':imu}
    print (param_dict)
    result = modelCD.create_code_module(
        phase=param_dict.pop('Phase', None),
        formula=param_dict.pop('Formula', None),
        params=param_dict, module_type=model_type, silent=True)
    print ('Component', elm, 'done!')

Build the code (ignore error messages generated by Cython regarding 'language_level')

In [ ]:
import pseudo_end
%cd ..

In [ ]:
elm_sys=sys_elems

In [ ]:
c = len(elm_sys)
c

In [ ]:
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
len(X)

In [ ]:
# XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
# XiXj

In [ ]:
mu = modelCD.mu
mu

In [ ]:
# Tsym = modelCD.get_symbol_for_t()

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
X.shape

In [ ]:
# dX = X[:,0]- bulk_comp[:,np.newaxis]
# dX

In [ ]:
# X0 = sym.MatrixSymbol('X_0',c,1)
# X0.shape

In [ ]:
# dX = (X[:,0]-X0[:,0])
# dX = X-X0
# dX.shape

In [ ]:
# sym.Matrix(dX)

In [ ]:
# if curv_method=='quad-full':
curv_string = ''
quad_strs = []
for i,j in cross_term_inds.T:
    # print(i, j)
    istr = 'k_' + str(i+1) + '_' + str(j+1)
    curv_string +=  istr + ' '
    quad_strs.append(istr)
    

quad_consts = sym.Matrix(list(sym.symbols(curv_string)))


In [ ]:
quad_consts

In [ ]:
# quad_strs

In [ ]:
XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
G_quad = nT*np.dot(XiXj, quad_consts)[0]

In [ ]:
G_quad

In [ ]:
mu_shft = sym.symbols('mu_shft')
mu_shft

In [ ]:
Ncross_terms = cross_term_inds.shape[1]
Ncross_terms

In [ ]:
Gshft = mu_shft*nT

In [ ]:
G = G_ss + G_quad + Gshft

In [ ]:
for istr, isym in zip(quad_strs, quad_consts):
    modelCD.add_expression_to_model(
        G, [(istr, 'J', isym)])

In [ ]:
modelCD.add_expression_to_model(G, [('mu_shft', 'J/m', mu_shft)])

In [ ]:
modelCD.module = "pseudo_soln"

In [ ]:
# dX = (X[:,0]-bulk_comp[:,np.newaxis])
# dX2 = np.dot(dX.T,dX)[0,0]
# dX2
# 
# G_mix, k_mix = sym.symbols('G_mix k_mix')

In [ ]:
# G_mix = k_mix*dX2
# G_mix

In [ ]:
# G = G_ss + G_mix

In [ ]:
# modelCD.add_expression_to_model(G, [('k_mix', 'none', k_mix)])

In [ ]:
# modelCD.module = "pseudo_soln"

In [ ]:
formula = ''
convert = []
test = []
if use_oxides_as_basis:
    for ind,elm in enumerate(elm_sys):
        ox_index = list(core.chem.oxide_props['oxides']).index(elm)
        ox_cat = core.chem.oxide_props['cations'][ox_index]
        formula += ox_cat + '[' + ox_cat + ']'
        ox_cat_num = core.chem.oxide_props['cat_num'][ox_index]
        if ox_cat_num > 1:
            convert.append('['+str(ind)+']=['+ox_cat+']/'+str(ox_cat_num)+'.0')
        else:
            convert.append('['+str(ind)+']=['+ox_cat+']')
        test.append('['+str(ind)+'] >= 0.0')
    formula += 'O[O]'
else:
    for ind,elm in enumerate(elm_sys):
        formula += elm + '[' + elm + ']'
        convert.append('['+str(ind)+']=['+elm+']')
        test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
mu_curv = np.tile(mu_curv/Ncross_terms, Ncross_terms)
mu_curv

In [ ]:
# curv_local

In [ ]:
paramValues = {'T_r':298.15,'P_r':1.0}

In [ ]:
for isym, ival in zip(quad_strs, mu_curv):
    paramValues[isym] = ival
    
paramValues['mu_shft'] = 1e3
    
endmembers = []
for elm in elm_sys:
    endmembers.append(str(elm)+'_pseudo_end')

In [ ]:
paramValues

In [ ]:
# paramValues = {'k_mix':curv_local,'T_r':298.15,'P_r':1.0,}


In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
# cd ..

In [ ]:
modelCD.create_code_module(
    phase="PseudoPhase", params=paramValues, endmembers=endmembers, 
    prefix="cy", module_type='calib', silent=False)

In [ ]:
import pseudo_soln
%cd ..

In [ ]:
#%cd working
#import pseudo_soln
#%cd ..
modelPseudo = model.Database(database="CoderModule", calib="calib", 
                         phase_tuple=('pseudo_soln', {'Psu':['PseudoPhase','solution']}))
Pseudo = modelPseudo.get_phase('Psu')

for phase_name, abbrv in zip(modelPseudo.phase_info.phase_name,modelPseudo.phase_info.abbrev):
    print ('Abbreviation: {0:<10s} Name: {1:<30s}'.format(abbrv, phase_name))

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
names = Pseudo.param_names

# Pseudo.set_param_values(param_names=[2],param_values=[1.0])

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
len(vals)

In [ ]:
# mu_bulk

In [ ]:
# mu_shft_val=+50e3
# mu_shft_val=+10e3
# mu_shft_val=+15e3
# mu_shft_val=+100e3
# mu_shft_val=+1000e3
mu_shft_val=-mu_bulk+5e3
# mu_shft_val=0
Pseudo.set_param_values(param_names=[len(vals)-1], param_values=[mu_shft_val])

In [ ]:
# vals[2:] *= 10

In [ ]:
# mu_curv

In [ ]:
# vals

In [ ]:
# for ind, ival in enumerate(vals[2:]):
#     Pseudo.set_param_values(param_names=[ind+2], param_values=[ival])

In [ ]:
# Pseudo.get_param_values(all_params=True)

Check pseudo-phase import by printning some phase characteristics

In [ ]:
print (Pseudo.props['phase_name'])
print (Pseudo.props['formula'])
print (Pseudo.props['molwt'])
print (Pseudo.props['abbrev'])
print (Pseudo.props['endmember_num'])
print (Pseudo.props['endmember_name'])

## Try the equiibrium calculations with the omnicomponent pseudo-phase
#### Choose a phase assemblage

In [ ]:
#stix_phases.keys()
phs_sys  = [Pseudo]

# phs_sys.extend(phases.values())

# phs_sys += [phases['Fsp'], phases['Ol'], phases['Cpx'], phases['Grt']] # solutiopns,
# phs_sys += [phases['Qz'], phases['Ky'], phases['Nph']]
#
#phs_sys  = [Pseudo, stix_phases['Opx']]

In [ ]:
for iphssym in hull_phs_sym:
    phs_sys.append(phases[iphssym])
    
    

In [ ]:
phs_sys

In [ ]:
# phases = get_subsolidus_phases(database=database)

In [ ]:
sys_elems
# phs_sys

In [ ]:
# equil = equilibrate.Equilibrate(['O','Na','Mg','Al','Si','Ca','Fe'], phs_sys)
equil = equilibrate.Equilibrate(sys_elems, phs_sys)

In [ ]:
mu_bulk

In [ ]:
1467815.52

In [ ]:
state = equil.execute(T, P, bulk_comp=bulk_comp, debug=0)
state.print_state()

In [ ]:
equil.element_list

In [ ]:
equil.moles_in

#### Set the bulk composition of the system
Input is a bulk peridotite suggested by Stixrude and Lithgow-Bertelloni

In [ ]:
grm_oxides = {
    'SiO2':  45.47, 
    'Al2O3':  4.0, 
    'FeO':    7.22, 
    'MgO':   38.53, 
    'CaO':    3.59, 
    'Na2O':   0.31
}
blk_cmp = np.zeros(7)
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
blk_cmp[0] = 2.0*mol_oxides[0] + 3.0*mol_oxides[2] + mol_oxides[5] + mol_oxides[7] + mol_oxides[10] + mol_oxides[11] # oxygen
blk_cmp[1] = 2.0*mol_oxides[11] # sodium
blk_cmp[2] = mol_oxides[7]      # magnesium
blk_cmp[3] = 2.0*mol_oxides[2]  # aluminum
blk_cmp[4] = mol_oxides[0]      # silicon
blk_cmp[5] = mol_oxides[10]     # calcium
blk_cmp[6] = mol_oxides[5]      # iron

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()

In [ ]:
grm_oxides = {
    'SiO2':  45.47, 
    'Al2O3':  4.0, 
    'FeO':    7.22, 
    'MgO':   38.53, 
    'CaO':    3.59, 
    'Na2O':   0.31
}
blk_cmp = np.zeros(7)
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
blk_cmp[0] = 2.0*mol_oxides[0] + 3.0*mol_oxides[2] + mol_oxides[5] + mol_oxides[7] + mol_oxides[10] + mol_oxides[11] # oxygen
blk_cmp[1] = 2.0*mol_oxides[11] # sodium
blk_cmp[2] = mol_oxides[7]      # magnesium
blk_cmp[3] = 2.0*mol_oxides[2]  # aluminum
blk_cmp[4] = mol_oxides[0]      # silicon
blk_cmp[5] = mol_oxides[10]     # calcium
blk_cmp[6] = mol_oxides[5]      # iron
blk_cmp

In [ ]:

sys_elems

In [ ]:
mass_oxides = {
    'SiO2':  45.47, 
    'Al2O3':  4.0, 
    'FeO':    7.22, 
    'MgO':   38.53, 
    'CaO':    3.59, 
    'Na2O':   0.31
}
mol_oxides = core.chem.format_mol_oxide_comp(mass_oxides, convert_grams_to_moles=True)

#elem_ind = {}
#elem_ind['O'] = np.find(sys_elems=='O')[0]
#bulk_comp = np.zeros(Nelems)
#bulk_comp[sys_elems=='Mg'] =1
#bulk_comp[sys_elems=='Si'] =1
#bulk_comp[sys_elems=='Al'] =1
#bulk_comp[sys_elems=='Fe'] =1
#bulk_comp[sys_elems=='Ca'] =1
#bulk_comp[sys_elems=='Na'] =1

In [ ]:
core.chem.OXIDE_ORDER

In [ ]:

bulk_elems = ['O','Na','Mg','Al','Si','Ca','Fe']

In [ ]:
elem_inds = {}
for elem in bulk_elems:
    elem_inds[elem] = np.where(sys_elems==elem)[0][0]

In [ ]:
elem_inds

In [ ]:
oxide_elems = {}
oxide_elems['SiO2'] = {'Si':1,'O':2}
oxide_elems['MgO'] = {'Mg':1,'O':1}
oxide_elems['Al2O3'] = {'Al':2,'O':3}
oxide_elems['CaO'] = {'Ca':1,'O':1}
oxide_elems['NaO'] = {'Na':1,'O':1}
oxide_elems['FeO'] = {'Fe':1,'O':1}

oxide_elems

In [ ]:
bulk_comp = np.zeros(Nelems)
for ox in mol_oxides:
    val = mol_oxides[ox]
    print(val)

In [ ]:

bulk_comp[sys_elems=='Mg'] =1
bulk_comp[sys_elems=='Si'] =1
bulk_comp[sys_elems=='Al'] =1

## Phases in Stixrude
Instantiate the database and optionally, print an info table

#### Create a Phase object for each phase in the database
Lode this information into dictionaries and lists for future reference

# Construct pseudophase

Aaron's notes:

1. Calculate the lower convex hull for the pure and endmember phases.
2. Adopt a modified ideal solution, where the mixing contribution is given by a scaled ideal entropy $-c RT \sum_i X_i \log X_i$.
3. Endmember chemical potentials as well as the scale factor $c$, are determined by least-squares fitting.
4. Endmember chemical potentials must be adjusted upwards to guarantee that the phase is everywhere metastable.
4.1 Endmember potentials are shifted so that every vertex of the convex hull lies on or below the omnicomponent surface, ensuring that all the equilibrium pure and endmember phases are individually stable relative to the omnicomponent phase.
4.2 Calculate center point of the hull vertices, add $X_i \sum_i \log X_i$ to the hull at thgis point, and insure that omnicomponent phase has an energy equal to or above this energy
5. Finally add an additional 1 J to each endmember potential just to insure numerical stability.

### (1) composition matrix
Relate endmember of each phase (rows) to moles of elements (columns):
- columns are indexed on atomic number
- rows are indexed on phase order, listed above

In [ ]:
C = []
row_names = []
for phase in pure_phases:
    if use_oxides_as_basis:
        C.append(core.chem.calc_mol_oxide_comp(phase.props['element_comp'][0]))
    else:
        C.append(phase.props['element_comp'][0])
    row_names.append(phase.props['endmember_name'][0])
for phase in soln_phases:
    for i in range(0,phase.endmember_num):
        if use_oxides_as_basis:
            C.append(core.chem.calc_mol_oxide_comp(phase.props['element_comp'][i]))
        else:
            C.append(phase.props['element_comp'][i])
        row_names.append(phase.props['endmember_name'][i])
C = np.array(C)
C.shape

Filter for the non-zero abundance elements/oxides in the system

In [ ]:
elm_sys_ind = np.where(np.sum(C,axis=0) > 0)[0]
if use_oxides_as_basis:
    elm_sys = [core.chem.oxide_props['oxides'][i] for i in elm_sys_ind]
else:
    elm_sys = [core.chem.PERIODIC_ORDER[i] for i in elm_sys_ind]
elm_sys

Deflate the composition matrix:
- columns correspond to non-zero elemental abundances in teh system
- rows are as previous

In [ ]:
C = C[:,elm_sys_ind]
C.shape

### (2) make a vector of chemical potentials of each endmember

In [ ]:
mu = []
for pureph in pure_phases:
    mu += [pureph.gibbs_energy(t,p)]
for solnph in soln_phases:
    mu += [solnph.gibbs_energy(t,p,mol=np.eye(solnph.endmember_num)[i],deriv={"dmol":1})[0,i] for i in range(0,solnph.endmember_num)]
mu = np.array(mu)
mu.shape

### (3) Convex Hull construction
- Hull construction depends ONLY on C and mu as defined above
- We define an additional multidimensional point called a viewpoint from which we can ask the question "what hull facets are viewable from that viewpoint?" 
- If the viewpoint is the bulk composition of the system, and we give the viewpoint an energy lower than any of the computed values of mu, then we should be able to "see" from that viewpoint all the hull facets that define the lowest energy polyhedron.
- The vertices of these facets are the "active" phases from which we can construct the pseudo-omnicomponent phase

For the system bulk composition, we use the average composition of the system, normalized to one mole.  
For the chemical potential we use 1.5 times the most negative chemical potential of any endmember in the system

#### Reduce the C matrix to unique rows (unique phase stoichiometry)
The c_inverse_array maps the indices of C_unique back to the full C matrix

In [ ]:
C_unique,C_inverse_ind = np.unique(C, axis=0,return_inverse=True)
C_unique.shape,C_inverse_ind.shape

Examine the chemical potentials of all stoichiometrically redundant phases and load an array with the most negative chemical potential (the stablest) of all the values found.

In [ ]:
mu_unique = []
for i in range(0,C_unique.shape[0]):
     mu_unique.append(np.min(mu[np.where(C_inverse_ind == i)]))
mu_unique = np.array(mu_unique)

Next, scale all the rows for one mole of each phase

In [ ]:
for i in range(0,C_unique.shape[0]):
    sum = np.sum(C_unique[i,:])
    C_unique[i,:] /= sum
    mu_unique[i] /= sum

Compute an average composition and assign it a chemical potentials 1.5 times more negative than the most negative unique endmember

In [ ]:
blk_cmp = np.sum(C_unique,axis=0)
blk_cmp = blk_cmp/np.sum(blk_cmp)
print ('Average composition of the viewpoint:', blk_cmp)
mu_blk_cmp = np.min(mu_unique)*1.5
print ('Chemical potential of the viewpoint:', mu_blk_cmp)

Now contruct the convex hull, using the extra bulk composition point as a viewpoint below the hull.  
The 'OJ' option is required to avoid roundoff errors that inhibit contruction; remove the option and run the code for a complete explanation.  
The 'OGn' option locates the viewpoint, which does not otherwise contribute to hull construction. 

In [ ]:
point_A = np.vstack((C_unique,blk_cmp))
point_B = np.vstack((np.reshape(mu_unique,(mu_unique.shape[0],1)),np.array([mu_blk_cmp])))
points = np.hstack((point_A,point_B))
hull = sci.spatial.ConvexHull(points, qhull_options='QJ QG'+str(points.shape[0]-1))

In [ ]:
hull.simplices.shape, hull.vertices.shape, hull.points.shape

### (4) Cull the hull
Determine which rows (phase endmembers) contribute to the lower most hull (as seen by the bulk composition)

In [ ]:
act_ind = np.full(mu_unique.shape, False, dtype=bool)
for visible_facet in hull.simplices[hull.good]:
    for index in visible_facet:
        act_ind[index] = True
for i,v in enumerate(act_ind):
    if not v:
        print(row_names[i], 'is not on the lower hull')
act_ind

Remove rows from C and mu, that is remove the phases that cannot be "seen" from the viewpoint

In [ ]:
mu = mu_unique[act_ind]
mu.shape

In [ ]:
C = C_unique[act_ind,:]
C.shape

### (5) solve for the internally consistent chemical potenials of the elements
Note that the endmember chemical potentials are adjusted by the pseudo-phase solution entropy.  This insures that the regression yields endmember potentials for the chemical elements that are consistent with ideal mixing.

In [ ]:
Cplus = np.hstack((C,np.zeros((C.shape[0],1))))
for i in range(0,C.shape[0]):
    sum = np.sum(C[i,:])
    s = 0.0
    for j in range(0,C.shape[1]):
        if C[i,j] > 0:
            X = C[i,j]/sum
            s += X*np.log(X)
    Cplus[i,-1] = s*sum*8.3143*t

The last regression parameter, $m$, is a multiplicity factor on the ideal entropy, while the other terms, $\mu _i^{o,elm}$, are the endmember chemical potentials of the pseudo-phase. The endmembers are the chemical elements.  I.e.,  
${\hat G^{pseudo}} = \sum\limits_i^{elm} {X_i^{elm}} \mu _i^{o,elm} + mRT\sum\limits_i^{elm} {X_i^{elm}\ln } X_i^{elm}$

In [ ]:
x,residuals,rank,s = np.linalg.lstsq(Cplus,mu,rcond=None)
x,np.sqrt(residuals),rank,s

Apply the offset to the endmember chemical potentials

In [ ]:
for i in range(0,x.shape[0]-1):
    x[i] += offset_value

## Build endmembers of pseudo-phase using the coder module

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
def standardize_formula(form):
    cmp = form.split('O')
    str = ''
    if cmp[0][-1].isdigit():
        str += cmp[0][:-1] + '(' + cmp[0][-1] + ')'
    else:
        str += cmp[0] + '(1)'
    if cmp[1] == '':
        str += 'O'
    else:
        str += 'O(' + cmp[1] + ')'
    return str

In [ ]:
model_type = "calib"
for ind,elm in enumerate(elm_sys):
    if use_oxides_as_basis:
        formula = standardize_formula(elm)
    else:
        formula = elm+'(1)'
    param_dict = {'Phase':elm,'Formula':formula,'T_r':298.15,'P_r':1.0,'GTP':x[ind]}
    print (param_dict)
    result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                      formula=param_dict.pop('Formula', None),
                                      params=param_dict,
                                      module_type=model_type,
                                      silent=True)
    print ('Component', elm, 'done!')

Build the code (ignore error messages generated by Cython regarding 'language_level')

In [ ]:
import pseudo_end
%cd ..

## Test the endmember code

In [ ]:
if test_endmember_code:
    print ('Endmember metadata:')
    try:
        print(pseudo_end.cy_Fe_pseudo_end_calib_identifier())
        print(pseudo_end.cy_Fe_pseudo_end_calib_name())
        print(pseudo_end.cy_Fe_pseudo_end_calib_formula())
        print(pseudo_end.cy_Fe_pseudo_end_calib_mw())
        print(pseudo_end.cy_Fe_pseudo_end_calib_elements())
    except AttributeError:
        pass
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    print ('Thermodynamic properties:')
    try:
        print(fmt.format('G', pseudo_end.cy_Fe_pseudo_end_calib_g(t,p), 'J/m'))
        print(fmt.format('dGdT', pseudo_end.cy_Fe_pseudo_end_calib_dgdt(t,p), 'J/K-m'))
        print(fmt.format('dGdP', pseudo_end.cy_Fe_pseudo_end_calib_dgdp(t,p), 'J/bar-m'))
        print(fmt.format('d2GdP2', pseudo_end.cy_Fe_pseudo_end_calib_d2gdt2(t,p), 'J/K^2-m'))
        print(fmt.format('d2GdTdP', pseudo_end.cy_Fe_pseudo_end_calib_d2gdtdp(t,p), 'J/K-bar-m'))
        print(fmt.format('d2GdP2', pseudo_end.cy_Fe_pseudo_end_calib_d2gdp2(t,p), 'J/bar^2-m'))
        print(fmt.format('d3GdT3', pseudo_end.cy_Fe_pseudo_end_calib_d3gdt3(t,p), 'J/K^3-m'))
        print(fmt.format('d3GdT2dP', pseudo_end.cy_Fe_pseudo_end_calib_d3gdt2dp(t,p), 'J/K^2-bar-m'))
        print(fmt.format('d3GdTdP2', pseudo_end.cy_Fe_pseudo_end_calib_d3gdtdp2(t,p), 'J/K-bar^2-m'))
        print(fmt.format('d3GdP3', pseudo_end.cy_Fe_pseudo_end_calib_d3gdp3(t,p), 'J/bar^3-m'))
        print(fmt.format('S', pseudo_end.cy_Fe_pseudo_end_calib_s(t,p), 'J/K-m'))
        print(fmt.format('V', pseudo_end.cy_Fe_pseudo_end_calib_v(t,p), 'J/bar-m'))
        print(fmt.format('Cv', pseudo_end.cy_Fe_pseudo_end_calib_cv(t,p), 'J/K-m'))
        print(fmt.format('Cp', pseudo_end.cy_Fe_pseudo_end_calib_cp(t,p), 'J/K-m'))
        print(fmt.format('dCpdT', pseudo_end.cy_Fe_pseudo_end_calib_dcpdt(t,p), 'J/K^2-m'))
        print(fmt.format('alpha', pseudo_end.cy_Fe_pseudo_end_calib_alpha(t,p), '1/K'))
        print(fmt.format('beta', pseudo_end.cy_Fe_pseudo_end_calib_beta(t,p), '1/bar'))
        print(fmt.format('K', pseudo_end.cy_Fe_pseudo_end_calib_K(t,p), 'bar'))
        print(fmt.format('Kp', pseudo_end.cy_Fe_pseudo_end_calib_Kp(t,p), ''))
    except AttributeError:
        pass
    print ('Parameters:')
    try:
        npar = pseudo_end.cy_Fe_pseudo_end_get_param_number()
        names = pseudo_end.cy_Fe_pseudo_end_get_param_names()
        units = pseudo_end.cy_Fe_pseudo_end_get_param_units()
        values = pseudo_end.cy_Fe_pseudo_end_get_param_values()
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except AttributeError:
        pass
    try:
        values[1] = 100.0
        pseudo_end.cy_Fe_pseudo_end_set_param_values(values)
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except (AttributeError, NameError):
        pass
    try:
        pseudo_end.cy_Fe_pseudo_end_set_param_value(1, 1.0)
        fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
        for i in range(0,npar):
            print(fmt.format(names[i], values[i], pseudo_end.cy_Fe_pseudo_end_get_param_value(i), units[i]))
    except AttributeError:
        pass

## Build solution pseudo-phase using the coder module
This code utilizes the previous endmember models generated above. 

In [ ]:
c = len(elm_sys)

In [ ]:
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
T = modelCD.get_symbol_for_t()
mu = modelCD.mu

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
S_config,R,multiplier = sym.symbols('S_config R multiplier')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= -R*nT*multiplier

In [ ]:
G_config = sym.simplify(-T*S_config)
G_config

In [ ]:
G = G_ss + G_config

In [ ]:
modelCD.add_expression_to_model(G, [('multiplier', 'none', multiplier)])

In [ ]:
modelCD.module = "pseudo_soln"

In [ ]:
formula = ''
convert = []
test = []
if use_oxides_as_basis:
    for ind,elm in enumerate(elm_sys):
        ox_index = list(core.chem.oxide_props['oxides']).index(elm)
        ox_cat = core.chem.oxide_props['cations'][ox_index]
        formula += ox_cat + '[' + ox_cat + ']'
        ox_cat_num = core.chem.oxide_props['cat_num'][ox_index]
        if ox_cat_num > 1:
            convert.append('['+str(ind)+']=['+ox_cat+']/'+str(ox_cat_num)+'.0')
        else:
            convert.append('['+str(ind)+']=['+ox_cat+']')
        test.append('['+str(ind)+'] >= 0.0')
    formula += 'O[O]'
else:
    for ind,elm in enumerate(elm_sys):
        formula += elm + '[' + elm + ']'
        convert.append('['+str(ind)+']=['+elm+']')
        test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
paramValues = {'multiplier':x[-1],'T_r':298.15,'P_r':1.0,}
endmembers = []
for elm in elm_sys:
    endmembers.append(str(elm)+'_pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
modelCD.create_code_module(phase="PseudoPhase", params=paramValues, endmembers=endmembers, 
                         prefix="cy", module_type='calib', silent=False)

In [ ]:
import pseudo_soln
%cd ..

## Test the solution
Characteristics of the solution

In [ ]:
if test_solution_code:
    mol = blk_cmp
    print ('Solution metadata:')
    try:
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_identifier())
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_name())
        print(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_formula(t,p,mol))
    except AttributeError:
        pass
    print ('Elemental conversion routines')
    try:
        e = np.zeros(106)
        sum = np.sum(mol)
        for index in range(0,c):
            end = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_elements(index)
            for i in range(0,106):
                e[i] += end[i]*mol[index]/sum
        nConv = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_moles(e)
        for i in range(0,c):
            print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
                i, mol[i]/sum, nConv[i], nConv[i]-mol[i]/sum))
        if not pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_test_moles(nConv):
            print ('Output of intrinsic composition calculation fails tests for permissible values.')
    except AttributeError:
        pass
    print ('Composition conversion routines')
    try:
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_tot_moles(mol))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_mole_frac(mol))
        e = pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_moles_to_elm(mol)
        print (e)
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_moles(e))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_tot_moles(e))
        print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_conv_elm_to_tot_grams(e))
    except AttributeError:
        pass
    print ('Simple thermodynamic functions')
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    try:
        print(fmt.format('G', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_g(t,p,mol), 'J'))
        print(fmt.format('dGdT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdt(t,p,mol), 'J/K'))
        print(fmt.format('dGdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdp(t,p,mol), 'J/bar'))
        print(fmt.format('d2GdT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdt2(t,p,mol), 'J/K^2'))
        print(fmt.format('d2GdTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdtdp(t,p,mol), 'J/K-bar'))
        print(fmt.format('d2GdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdp2(t,p,mol), 'J/bar^2'))
        print(fmt.format('d3GdT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdt3(t,p,mol), 'J/K^3'))
        print(fmt.format('d3GdT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdt2dp(t,p,mol), 'J/K^2-bar'))
        print(fmt.format('d3GdTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdtdp2(t,p,mol), 'J/K-bar^2'))
        print(fmt.format('d3GdP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdp3(t,p,mol), 'J/bar^3'))
        print(fmt.format('S', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_s(t,p,mol), 'J/K'))
        print(fmt.format('V', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_v(t,p,mol), 'J/bar'))
        print(fmt.format('Cv', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_cv(t,p,mol), 'J/K'))
        print(fmt.format('Cp', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_cp(t,p,mol), 'J/K'))
        print(fmt.format('dCpdT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dcpdt(t,p,mol), 'J/K^2'))
        print(fmt.format('alpha', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_alpha(t,p,mol), '1/K'))
        print(fmt.format('beta', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_beta(t,p,mol), '1/bar'))
        print(fmt.format('K', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_K(t,p,mol), 'bar'))
        print(fmt.format('Kp', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_Kp(t,p,mol), ''))
    except AttributeError:
        pass
    print ('Endmember properties')
    fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
    try:
        print ("number of components", pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_number())
        for index in range(0, c):
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_name(index)), end=' ')
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_formula(index)), end=' ')
            print ("mw: {0:10.2f}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_mw(index)))
            print (fmt.format('mu0', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_mu0(index,t,p), 'J/mol'))
            print (fmt.format('dmu0dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
            print (fmt.format('dmu0dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
            print (fmt.format('d2mu0dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
            print (fmt.format('d2mu0dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
            print (fmt.format('d2mu0dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
            print (fmt.format('d3mu0dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
            print (fmt.format('d3mu0dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
            print (fmt.format('d3mu0dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
            print (fmt.format('d3mu0dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
            print ("Element array:")
            print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_endmember_elements(index))
            print ()
    except AttributeError:
        pass
    print ('Species properties:')
    fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
    try:
        print ("number of species", pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_number())
        for index in range(0, c):
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_name(index)), end=' ')
            print ("{0:<20.20s}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_formula(index)), end=' ')
            print ("mw: {0:10.2f}".format(pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_mw(index)))
            print ("Element array:")
            print (pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_species_elements(index))
            print ()
    except AttributeError:
        pass
    print ('First compositional derivatines:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:13.6e}".format(x), end=' ') for x in result]
        print ("{0:<10.10s}".format(units))
    def printLabels(n):
        print ("{0:<18.18s}".format(''), end=' ')
        [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
        print ()
    printLabels(mol)
    try:
        printResult('dGdn', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_dgdn(t,p,mol), 'J/m')
        printResult('d2GdndT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdndt(t,p,mol), 'J/K-m')
        printResult('d2GdndP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdndp(t,p,mol), 'J/bar-m')
        printResult('d3GdndT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndt2(t,p,mol), 'J/K^2-m')
        printResult('d3GdndTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndtdp(t,p,mol), 'J/K-bar-m')
        printResult('d3GdndP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdndp2(t,p,mol), 'J/bar^2-m')
        printResult('d4GdndT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndt3(t,p,mol), 'J/K^3-m')
        printResult('d4GdndT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndt2dp(t,p,mol), 'J/K^2-bar-m')
        printResult('d4GdndTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndtdp2(t,p,mol), 'J/K-bar^2-m')
        printResult('d4GdndP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdndp3(t,p,mol), 'J/bar^3-m')
    except AttributeError:
        pass 
    print ('Second compositional derivatives:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:13.6e}".format(x), end=' ') for x in result]
        print ("{0:<10.10s}".format(units))
    def printLabels(n):
        print ("{0:<18.18s}".format(''), end=' ')
        maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
        [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
        print ()
    printLabels(mol)
    try:
        printResult('d2Gdn2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d2gdn2(t,p,mol), 'J/m^2')
        printResult('d3Gdn2dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn2dt(t,p,mol), 'J/K-m^2')
        printResult('d3Gdn2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn2dp(t,p,mol), 'J/bar-m^2')
        printResult('d4Gdn2dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dt2(t,p,mol), 'J/K^2-m^2')
        printResult('d4Gdn2dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dtdp(t,p,mol), 'J/K-bar-m^2')
        printResult('d4Gdn2dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn2dp2(t,p,mol), 'J/bar^2-m^2')
        printResult('d5Gdn2dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dt3(t,p,mol), 'J/K^3-m^2')
        printResult('d5Gdn2dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dt2dp(t,p,mol), 'J/K^2-bar-m^2')
        printResult('d5Gdn2dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dtdp2(t,p,mol), 'J/K-bar^2-m^2')
        printResult('d5Gdn2dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn2dp3(t,p,mol), 'J/bar^3-m^2')
    except AttributeError:
        pass
    print ('Third compositional derivatives:')
    def printResult(name, result, units):
        print ("{0:<10.10s}".format(name), end=' ')
        [print ("{0:10.3e}".format(x), end=' ') for x in result]
        print ("{0:<14.14s}".format(units))
    def printLabels(n):
        print ("{0:<15.15s}".format(''), end=' ')
        maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
        [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
        print ()
    printLabels(mol)
    try:
        printResult('d3Gdn3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d3gdn3(t,p,mol), 'J/m^3')
        printResult('d4Gdn3dT', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn3dt(t,p,mol), 'J/K-m^3')
        printResult('d4Gdn3dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d4gdn3dp(t,p,mol), 'J/bar-m^3')
        printResult('d5Gdn3dT2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dt2(t,p,mol), 'J/K^2-m^3')
        printResult('d5Gdn3dTdP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dtdp(t,p,mol), 'J/K-bar-m^3')
        printResult('d5Gdn3dP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d5gdn3dp2(t,p,mol), 'J/bar^2-m^3')
        printResult('d6Gdn3dT3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dt3(t,p,mol), 'J/K^3-m^3')
        printResult('d6Gdn3dT2dP', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dt2dp(t,p,mol), 'J/K^2-bar-m^3')
        printResult('d6Gdn3dTdP2', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dtdp2(t,p,mol), 'J/K-bar^2-m^3')
        printResult('d6Gdn3dP3', pseudo_soln.cy_PseudoPhase_pseudo_soln_calib_d6gdn3dp3(t,p,mol), 'J/bar^3-m^3')
    except AttributeError:
        pass

## Import model for the pseudo-phase into the ThermoEngine package

In [ ]:
#%cd working
#import pseudo_soln
#%cd ..
modelPseudo = model.Database(database="CoderModule", calib="calib", 
                         phase_tuple=('pseudo_soln', {'Psu':['PseudoPhase','solution']}))
Pseudo = modelPseudo.get_phase('Psu')

for phase_name, abbrv in zip(modelPseudo.phase_info.phase_name,modelPseudo.phase_info.abbrev):
    print ('Abbreviation: {0:<10s} Name: {1:<30s}'.format(abbrv, phase_name))

Check pseudo-phase import by printning some phase characteristics

In [ ]:
print (Pseudo.props['phase_name'])
print (Pseudo.props['formula'])
print (Pseudo.props['molwt'])
print (Pseudo.props['abbrev'])
print (Pseudo.props['endmember_num'])
print (Pseudo.props['endmember_name'])

## Try the equiibrium calculations with the omnicomponent pseudo-phase
#### Choose a phase assemblage

In [ ]:
#stix_phases.keys()
phs_sys  = [Pseudo]
phs_sys += [stix_phases['Fsp'], stix_phases['Ol'], stix_phases['Cpx'], stix_phases['Grt']] # solutiopns,
phs_sys += [stix_phases['Qz'], stix_phases['Ky'], stix_phases['Nph']]
#
#phs_sys  = [Pseudo, stix_phases['Opx']]

In [ ]:
equil = equilibrate.Equilibrate(['O','Na','Mg','Al','Si','Ca','Fe'], phs_sys)

#### Set the bulk composition of the system
Input is a bulk peridotite suggested by Stixrude and Lithgow-Bertelloni

In [ ]:
grm_oxides = {
    'SiO2':  45.47, 
    'Al2O3':  4.0, 
    'FeO':    7.22, 
    'MgO':   38.53, 
    'CaO':    3.59, 
    'Na2O':   0.31
}
blk_cmp = np.zeros(7)
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
blk_cmp[0] = 2.0*mol_oxides[0] + 3.0*mol_oxides[2] + mol_oxides[5] + mol_oxides[7] + mol_oxides[10] + mol_oxides[11] # oxygen
blk_cmp[1] = 2.0*mol_oxides[11] # sodium
blk_cmp[2] = mol_oxides[7]      # magnesium
blk_cmp[3] = 2.0*mol_oxides[2]  # aluminum
blk_cmp[4] = mol_oxides[0]      # silicon
blk_cmp[5] = mol_oxides[10]     # calcium
blk_cmp[6] = mol_oxides[5]      # iron

In [ ]:
state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0)
state.print_state()